In [1]:
# Dependencies
import math
import pymongo


import time 
from bs4 import BeautifulSoup
from splinter import Browser

from geopy.geocoders import Nominatim


In [2]:
# make browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
data_years=[i for i in range(2007,2013)]
data_years
data_count=[]
pages={}
totalRecruits=0
for i in data_years:
    url='https://247sports.com/Season/' +str(i)+ '-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=1'
    browser.visit(url)
    html=browser.html
    soup  = BeautifulSoup(html, 'html.parser')
    recruits = int(soup.find('section', class_="list-page").span.text.split("(")[1].split(')')[0])
    totalRecruits = totalRecruits +  recruits
    pages[i]=math.ceil(recruits/50)
    print(totalRecruits)
    print(pages)

2214
{2007: 45}
4382
{2007: 45, 2008: 44}
6673
{2007: 45, 2008: 44, 2009: 46}
9134
{2007: 45, 2008: 44, 2009: 46, 2010: 50}
11813
{2007: 45, 2008: 44, 2009: 46, 2010: 50, 2011: 54}
14915
{2007: 45, 2008: 44, 2009: 46, 2010: 50, 2011: 54, 2012: 63}


In [6]:
totalRecruits

14915

In [7]:
urlDir={}
for i in data_years:
    pageNums=pages[i]
    urlL=[]
    for j in range (pageNums):
        url='https://247sports.com/Season/' +str(i)+ '-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page='+str(j+1)
        urlL.append(url)
    urlDir[i]=urlL
urlDir


{2007: ['https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=1',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=2',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=3',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=4',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=5',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=6',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=7',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=8',
  'https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=9',
  'https://247sports.c

In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.sports247RecruitRanks
collection = db.sports247RecruitData

In [9]:
# make browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [10]:
# loop over the dir to extract data
for data_year in data_years[:]:
    urlExtracted=[]
    count=0
    print(data_year)
    for url in urlDir[data_year]:
        recruits={}
        player=[]
        school=[]
        city=[]
        state=[]
        cityState=[]
        position=[]
        height=[]
        heightFt=[]
        heightIn=[]
        weight=[]
        rate=[]
        rankP=[]
        scoreL=[]
        colleges=[]
        years=[]
        browser.visit(url)
        print(count)
        count=count+1
        

        html = browser.html
        soup  = BeautifulSoup(html, 'html.parser')

        name=soup.find_all('div', class_='name')
        positions = soup.find_all('li', class_="position")
        heights = soup.find_all('li', class_="height")
        weights = soup.find_all('li', class_="weight")
        ratings = soup.find_all('div', class_='rating')
        rankPrimary = soup.find_all('div', class_="primary")
        scores = soup.find_all('span', class_="score")
        college = soup.find_all('div', class_="right-content")



        for i in range(1,len(name)):
            player.append(name[i].a.text)
            school.append(name[i].span.text.split(' (')[0].strip())
            meta = name[i].span.text.split(' (')[-1]
            city.append(meta.split(',')[0])
            state.append(meta.split(',')[-1].split(')')[0].strip())

            # geolocation lat long 
            stateCity=name[i].span.text.split(" (")[-1].split(")")[0]

            stateCity = stateCity +", united states of america"
            cityState.append(stateCity)


        for i in range(len(positions)):
            position.append(positions[i].text)

        for i in range(len(heights)):
            height.append(heights[i].text)
            heightFt.append(height[i].split('-')[0])
            heightIn.append(height[i].split('-')[-1])

        for i in range(len(weights)):
            wt=weights[i].text
            weight.append(wt)

        for i in range(1,len(ratings)):
            stars = len(ratings[i].find_all('span', class_="icon-starsolid yellow"))
            rate.append(stars)

        for i in range(len(rankPrimary)):
            rank = int(rankPrimary[i].text.split(' ')[1])
            rankP.append(rank)

        for i in range(len(scores)):
            score=scores[i].text
            scoreL.append(score)
            
        
        for i in range(1,len(college)):
            try:
                colg = college[i].find('img', class_='jsonly')['title']
                colleges.append(colg)
            except:
                colleges.append('NoCollege')
            

        for i in range(len(scores)):
            years.append(data_year)

        time.sleep(3)
        recruits={'player'  :player,
                  'city'    :city,
                  'state'   :state,
                  'school'  :school,
                  'location':cityState,
                  'position':position,
                  'heightFt':height,
                  'heightFt':heightFt,
                  'heightIn':heightIn,
                  'weight'  :weight,
                  'rate'    :rate,
                  'rank'    :rankP,
                  'score'   :scoreL,
                  'college' :colleges,
                  'year'    :years
                 }
        collection.insert_one(recruits)
        print(url)
        urlExtracted.append(url)
        urlExtracedSave=urlExtracted.copy()

    time.sleep(20)

2007
0
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=1
1
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=2
2
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=3
3
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=4
4
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=5
5
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=6
6
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=7
7
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=8
8
https://247sports.com/Season/2007-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=9
9
https://247sports.com/Season/2007-Football/Compo

32
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=33
33
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=34
34
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=35
35
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=36
36
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=37
37
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=38
38
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=39
39
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=40
40
https://247sports.com/Season/2008-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=41
41
https://247sports.com/Season/2008-

19
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=20
20
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=21
21
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=22
22
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=23
23
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=24
24
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=25
25
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=26
26
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=27
27
https://247sports.com/Season/2010-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=28
28
https://247sports.com/Season/2010-

46
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=47
47
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=48
48
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=49
49
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=50
50
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=51
51
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=52
52
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=53
53
https://247sports.com/Season/2011-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=54
2012
0
https://247sports.com/Season/2012-Football/CompositeRecruitRankings?InstitutionGroup=HighSchool&Page=1
1
https://247sports.com/Season/201